In [5]:
import pandas as pd
import json
import numpy as np

In [2]:
DATA_DIR = '../data/'
TWEET_DIR = '../data/tweets/'
events = open(DATA_DIR + 'event_names.txt', 'r').read().splitlines()

In [13]:
data_filter_method = '_noRT' # make sure to include underscore
cluster_method = ''

In [8]:
time_dict = json.load(open(DATA_DIR + "event_year.json","r"))
polarization_dict = json.load(open(DATA_DIR + "polarization"+data_filter_method+".json","r"))

In [9]:
x = []
y = []
y_random = []
labels=[]
ex = ['fort_lauderdale']  # we exclude Fort Lauderdale because we only have data for the first day after the shooting
for e, t in time_dict.items():
    if e in ex:
        continue
    label = e.split('_')
    new_label = []
    for l in label:
        new_label.append(l[0].upper() + l[1:])
    new_label = ' '.join(new_label)
    
    x_val = float(2000+t)
    y_val = float(polarization_dict[e][0])
    y_random_val = float(float(polarization_dict[e][1]))
    x.append(x_val)
    y.append(y_val)
    y_random.append(y_random_val)
    labels.append(new_label)
    #labels.append(plt.text(x_val, y_val, new_label, fontsize=8))
df = pd.DataFrame.from_dict({'year':np.array(x * 2), 'polarization':np.array(y + y_random), 'label':labels * 2, 'is_actual':['actual value']* len(y) + ['value resulting from random party assignment']* len(y)})
df.to_csv(DATA_DIR + "polarization"+data_filter_method+".csv")


In [14]:
between_topic_pol = json.load(open(DATA_DIR + "between_topic_polarization"+cluster_method+".json","r"))